<a href="https://colab.research.google.com/github/jdmartinev/ST1613-AppliedML-/blob/main/Semana07/Harry_Potter_Expert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q langchain
!pip install -q torch
!pip install -q transformers
!pip install -q sentence-transformers
!pip install -q datasets
!pip install -q faiss-cpu
!pip install -q pypdf
!pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [ ]:
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import AutoTokenizer, pipeline
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA
import torch
from langchain import PromptTemplate, LLMChain

from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    pipeline,
    AutoModelForCausalLM
    )


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

def print_lines(text, max_ch = 50):
  current_line = ""
  words = text.split()
  i = 0
  while i < len(words):
    if len(current_line) > 50:
      print(current_line)
      current_line = ""
    else:
      current_line += f"{words[i]} "
      i+=1
  if current_line:
    print(current_line)

cuda


# Datasets

In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1YcNlffQl6E09Erst__EZvlC0iJRgy6LF' -O libros.zip
!unzip libros.zip -d ./libros

--2024-05-14 16:37:48--  https://docs.google.com/uc?export=download&id=1YcNlffQl6E09Erst__EZvlC0iJRgy6LF
Resolving docs.google.com (docs.google.com)... 64.233.170.100, 64.233.170.102, 64.233.170.113, ...
Connecting to docs.google.com (docs.google.com)|64.233.170.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1YcNlffQl6E09Erst__EZvlC0iJRgy6LF&export=download [following]
--2024-05-14 16:37:48--  https://drive.usercontent.google.com/download?id=1YcNlffQl6E09Erst__EZvlC0iJRgy6LF&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.251.12.132, 2404:6800:4003:c11::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.251.12.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20809441 (20M) [application/octet-stream]
Saving to: ‘libros.zip’

libros.zip          100%[===================>]  19.84M  30.8MB/s    in 0.

## Leer documentos

In [ ]:
import os
path_docs = "./libros"

os.listdir(path_docs)

['Harry Potter - Book 7 - The Deathly Hallows.pdf',
 'Harry Potter - Book 5 - The Order of the Phoenix.pdf',
 'Harry Potter - Book 6 - The Half-Blood Prince.pdf',
 'Harry Potter - Book 1 - The Sorcerers Stone.pdf',
 'Harry Potter - Book 3 - The Prisoner of Azkaban.pdf',
 'Harry Potter - Book 2 - The Chamber of Secrets.pdf',
 'Harry Potter - Book 4 - The Goblet of Fire.pdf']

In [ ]:
list_pdf = [x for x in os.listdir(path_docs) if x.endswith(".pdf")]

In [ ]:
# Document Transformers
from tqdm.notebook import tqdm
from langchain.document_loaders import PyPDFLoader

chunk_size = 1000
chunk_overlap = 200

all_docs = []
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
    )

for name in tqdm(list_pdf):
  if name.endswith(".pdf"):
    path_tmp = os.path.join(path_docs, name)
    loader = PyPDFLoader(path_tmp)
    text = loader.load()
    chunks_of_text = text_splitter.split_documents(text)
    all_docs.extend(chunks_of_text)


  0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
from pprint import pprint
chunk = all_docs[100]

print_lines(chunk.page_content)

pprint(chunk.metadata)

only ones the Trace can’t detect, because we don’t 
need to cast spells to use them: brooms, thestrals, 
and Hagrid’s motorbike.” Harry could see ﬂaws in this 
plan; however, he held his tongue to give Mad-Eye the 
chance to address them. “Now, your mother’s charm will 
only break under two condi- tions: when you come of 
age, or”—Moody gestured around the pristine kitchen—“you 
no longer call this place home. You and your aunt and 
uncle are going your separate ways tonight, in the 
full understanding that you’re never going to live 
together again, correct?” Harry nodded. “So this time, 
when you leave, there’ll be no going back, and the 
charm will break the moment you get outside its range. 
We’ve choosing to break it early, because the alternative 
is waiting for 47 
{'page': 54,
 'source': './libros/Harry Potter - Book 7 - The Deathly Hallows.pdf'}


# Database

## Embeddings

In [ ]:
def get_embeddings_model(model_path=None):
  modelPath = "sentence-transformers/all-MiniLM-l6-v2"
  device = torch.device(
      "cuda" if torch.cuda.is_available() else "cpu"
      )
  #device = "cpu"
  if model_path:
    modelPath = model_path
  model_kwargs = {'device':device}
  encode_kwargs = {'normalize_embeddings': False}

  embeddings = HuggingFaceEmbeddings(
      model_name=modelPath,     # Provide the pre-trained model's path
      model_kwargs=model_kwargs, # Pass the model configuration options
      encode_kwargs=encode_kwargs # Pass the encoding options
  )
  print(f"device: {device}")
  return embeddings


In [ ]:
embeddings = get_embeddings_model()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


device: cuda


In [ ]:
text = "This is a test document."
query_result = embeddings.embed_query(text)
len(query_result)

384

## Vectore Store

In [ ]:
print(len(all_docs))

8927


In [ ]:
db = FAISS.from_documents(all_docs, embeddings)

In [ ]:
question = "Who is Mr. Dursley?"
searchDocs = db.similarity_search(question)
print_lines(searchDocs[0].page_content)

CHAPTER ONE THE BOY WHO LIVED M r. and Mrs. Dursley, 
of number four, Privet Drive, were proud to say that 
they were perfectly normal, thank you very much. They 
were the last people you’d expect to be involved in 
anything strange or mysterious, because they just didn’t 
hold with such nonsense. Mr. Dursley was the director 
of a firm called Grunnings, which made drills. He was 
a big, beefy man with hardly any neck, although he 
did have a very large mustache. Mrs. Dursley was thin 
and blonde and had nearly twice the usual amount of 
neck, which came in very useful as she spent so much 
of her time craning over garden fences, spying on the 
neighbors. The Dursleys had a small son called Dudley 
and in their opinion there was no finer boy anywhere. 
The Dursleys had everything they wanted, but they also 
had a secret, and their greatest fear was that somebody 
would discover it. They didn’t think they could bear 
it if anyone found out about the Potters. Mrs. Potter 
was Mrs. 


In [ ]:
import re
def get_context(db, question, top_k = 2):
  searchDocs = db.similarity_search(question, k= top_k)
  return re.sub(r"\t+", " ", "\n".join([x.page_content for x in searchDocs]))

In [ ]:
get_context(db, question)

'“Magic?” he repeated in a whisper. \n“That’s right,” said Dumbledore. \n“It’s … it’s magic, what I can do?” \n“What is it that you can do?” \n“All sorts,” breathed Riddle. A flush of excitement was \nrising up his neck into his hollow cheeks; he looked \nfevered. “I can make things move without touching \nthem. I can make animals do what I want them to do, \nwithout training them. I can make bad things happen \nto people who annoy me. I can make them hurt if I \nwant to.”\nmanage ter get yer hands on summat.”\n Harry sat and thought about this while Hagrid read his newspaper, the\nDaily Prophet. Harry had learned from Uncle Vernon that people liked to be left\nalone while they did this, but it was very difficult, he’d never had so many\nquestions in his life.\n “Ministry o’ Magic messin’ things up as usual,” Hagrid muttered, turning\nthe page.\n “There’s a Ministry of Magic?” Harry asked, before he could stop\nhimself.\n “’Course,” said Hagrid. “They wanted Dumbledore fer Minister, o’

# Large Language Model (LLM)

## Modelo a usar

In [ ]:
import torch
from transformers import AutoModelForQuestionAnswering
from transformers import TFAutoModelForQuestionAnswering
models_hf = ["deepset/minilm-uncased-squad2"]
model_ckpt = models_hf[0]

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained(model_ckpt)
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)


Some weights of the model checkpoint at deepset/minilm-uncased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
question = "Who is Mr. Dursley?"
context = get_context(db, question)
inputs = tokenizer(question, context, return_tensors="pt")
pipe = pipeline("question-answering", model=model, tokenizer=tokenizer)
pipe(question=question, context=context, topk=3)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/question_answering.py:326: UserWarning: topk parameter is deprecated, use top_k instead
  warnings.warn("topk parameter is deprecated, use top_k instead", UserWarning)


[{'score': 0.3200392425060272,
  'start': 318,
  'end': 357,
  'answer': 'the director of a firm called Grunnings'},
 {'score': 0.13510459661483765,
  'start': 322,
  'end': 357,
  'answer': 'director of a firm called Grunnings'},
 {'score': 0.059819769114255905,
  'start': 318,
  'end': 376,
  'answer': 'the director of a firm called Grunnings, which made\ndrills'}]